Basics
====

Let's first take a look at what's inside the ``ib_insync`` package:

In [1]:
import ib_insync
print(ib_insync.__all__)

['util', 'Object', 'ContractDetails', 'ContractDescription', 'ComboLeg', 'UnderComp', 'OrderComboLeg', 'OrderState', 'OrderStatus', 'ScannerSubscription', 'SoftDollarTier', 'Execution', 'CommissionReport', 'ExecutionFilter', 'BarData', 'RealTimeBar', 'HistogramData', 'NewsProvider', 'DepthMktDataDescription', 'AccountValue', 'RealTimeBar', 'TickData', 'MktDepthData', 'BracketOrder', 'DOMLevel', 'Trade', 'TradeLogEntry', 'ScanData', 'TagValue', 'PortfolioItem', 'Position', 'Fill', 'OptionComputation', 'OptionChain', 'NewsArticle', 'HistoricalNews', 'NewsTick', 'NewsBulletin', 'ConnectionStats', 'Contract', 'Stock', 'Option', 'Future', 'Forex', 'Index', 'CFD', 'Commodity', 'Bond', 'FuturesOption', 'MutualFund', 'Warrant', 'Order', 'LimitOrder', 'MarketOrder', 'StopOrder', 'StopLimitOrder', 'Ticker', 'IB', 'Client', 'Wrapper']


The following stanza imports everything and uses the Qt event loop to keep the notebook live updated:

In [2]:
%gui qt5

from ib_insync import *
util.useQt()

The main object of the package is "IB". Let's create an IB instance and connect to a running TWS/IBG application (remember it must have an open API port):

In [3]:
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=10)

If the connection failed, then double-check the hostname and port. For IB Gateway the default port is 4001. Make sure the clientId is not already in use.

If the connection succeeded, then ``ib`` will be synchronized with TWS/IBG. The "current state" is now available via various methods:

In [4]:
print("portolio:", ib.portfolio())
print("positions:", ib.positions())
print("trades:", ib.trades())
print("openTrades:", ib.openTrades())
print("orders:", ib.orders())
print("openOrders:", ib.openOrders())
print("fills:", ib.fills())
print("executions:", ib.executions())
print("nwesTicks:", ib.newsTicks())
print("managedAccounts:", ib.managedAccounts())
print("accountValues:", ib.accountValues())
print("tickers:", ib.tickers())

portolio: []
positions: []
trades: []
openTrades: []
orders: []
openOrders: []
fills: []
executions: []
nwesTicks: []
managedAccounts: ['DU15059']
accountValues: [AccountValue(account='DU15059', tag='AccountCode', value='DU15059', currency=''), AccountValue(account='DU15059', tag='AccountOrGroup', value='DU15059', currency='BASE'), AccountValue(account='DU15059', tag='AccountOrGroup', value='DU15059', currency='USD'), AccountValue(account='DU15059', tag='AccountReady', value='true', currency=''), AccountValue(account='DU15059', tag='AccountType', value='UNIVERSAL', currency=''), AccountValue(account='DU15059', tag='AccruedCash', value='0', currency='BASE'), AccountValue(account='DU15059', tag='AccruedCash', value='0', currency='USD'), AccountValue(account='DU15059', tag='AccruedCash-C', value='0.00', currency='USD'), AccountValue(account='DU15059', tag='AccruedCash-S', value='0.00', currency='USD'), AccountValue(account='DU15059', tag='AccruedDividend', value='0.00', currency='USD'), A

This state will automatically be kept in sync with TWS/IBG. For example, an execution will be added as soon as it is reported, or account values will be updated as soon as they change in TWS. To see some of the live updates in the notebook:

In [5]:
util.logToConsole()

This will put log messages under the current active cell.

To see all debug messages (including network traffic):

In [9]:
import logging
util.logToConsole(logging.DEBUG)

The IB class has nearly all request methods that the IB API offers.
The methods that return a result will block until finished and then return the result.
Take for example reqContractDetails:

In [5]:
contract = Stock('TSLA', 'SMART', 'USD')
%time cds = ib.reqContractDetails(contract)

cds

CPU times: user 4.45 ms, sys: 2.13 ms, total: 6.58 ms
Wall time: 125 ms


[ContractDetails(summary=Contract(conId=76792991, symbol='TSLA', secType='STK', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='TSLA', tradingClass='NMS'), marketName='NMS', minTick=0.01, orderTypes='ACTIVETIM,ADJUST,ALERT,ALGO,ALLOC,AON,AVGCOST,BASKET,COND,CONDORDER,DARKPOLL,DAY,DEACT,DEACTDIS,DEACTEOD,DIS,GAT,GTC,GTD,GTT,HID,IBDARK,ICE,IMB,IOC,LIT,LMT,LOC,MIT,MKT,MOC,MTL,NONALGO,OCA,OPG,OPGREROUT,PEGBENCH,POSTONLY,PREOPGRTH,REL,RPI,RTH,SCALE,SCALERST,STP,STPLMT,SWEEP,TIMEPRIO,TRAIL,TRAILLIT,TRAILLMT,TRAILMIT,WHATIF', validExchanges='SMART,CHX,ARCA,ISLAND,DRCTEDGE,NSX,BEX,BATS,EDGEA,BYX,PSX', priceMagnifier=1, longName='TESLA INC', industry='Consumer, Cyclical', category='Auto Manufacturers', subcategory='Auto-Cars/Light Trucks', timeZoneId='EST5EDT', tradingHours='20170717:0005-2358;20170718:0005-2358;20170719:0005-2358;20170720:0005-2358;20170721:0005-2358;20170722:0005-2358;20170723:0005-2358;20170724:0005-2358;20170725:0005-2358;20170726:0005-2358;20170727

It generally takes about 100 - 300 ms.

To create contracts, the standard IBAPI way is to do

In [6]:
contract = Contract()
contract.secType = 'STK'
contract.symbol = 'INTC'
contract.exchange = 'SMART'
contract.currency = 'USD'
contract.primaryExchange = 'NASDAQ'

contract

Contract(symbol='INTC', secType='STK', exchange='SMART', primaryExchange='NASDAQ', currency='USD')

The more convenient way to do this in ``ib_insync`` is

In [7]:
contract = Stock('INTC', 'SMART', 'USD', primaryExchange='NASDAQ')

contract

Stock(symbol='INTC', exchange='SMART', primaryExchange='NASDAQ', currency='USD')

A forex contract can be specified by it's common name:

In [8]:
contract = Forex('USDJPY')

contract

Forex('USDJPY', exchange='IDEALPRO')

An option:

In [9]:
Option('EOE', '20170721', 505, 'C', 'FTA', multiplier=100)

Option(symbol='EOE', lastTradeDateOrContractMonth='20170721', strike=505, right='C', multiplier=100, exchange='FTA')

The whole list of different of contract types:

In [10]:
ib_insync.contract.__all__

['Contract',
 'Stock',
 'Option',
 'Future',
 'Forex',
 'Index',
 'CFD',
 'Commodity',
 'Bond',
 'FuturesOption',
 'MutualFund',
 'Warrant']

For Order too are there some subtypes defined for convenience:

In [11]:
ib_insync.order.__all__

['Order', 'LimitOrder', 'MarketOrder', 'StopOrder', 'StopLimitOrder']

For example to create a limit order:

In [12]:
order = LimitOrder('BUY', 100, 12.34)

order

LimitOrder(action='BUY', totalQuantity=100, lmtPrice=12.34)

End the session:

In [13]:
ib.disconnect()